In [ ]:
import re
import json
from timeout_decorator import timeout
import os

In [ ]:
folder_name = 'gpt35'
files = os.listdir(f'../Generation/Output/{folder_name}/')
json_files = [file for file in files if file.endswith('128.json')]
print(json_files)

In [ ]:
@timeout(60)
def match_expression(expression, text):
    try:
        return re.search(expression, text) is not None
    except Exception as e:
        # print(expression,": ", e)
        return None

In [ ]:
for name in json_files:
    print("Starting file: ", name)
    name = name.replace('.json', '')
    filename = f'../Generation/Output/{folder_name}/{name}.json'
    outfilename = f"./Output/{folder_name}/{name}_Compiled_Result.json"
    with open(filename, 'r') as f:
        data = json.load(f)
    print(len(data))

    compilation_result = []
    for item in data:
        # print("Starting item: ", item['id'])
        id = item['id']
        matches = item['matches']
        non_matches = item['non_matches']
        # for completion in item['t5_output']:
        #     expression = completion['generated_text']
        # for i in range(len(item['output'])):
        #     expression = item['output'][i]['text']
        for i in range(len(item['output']['choices'])):
            expression = item['output']['choices'][i]['message']['content']
            item['completion'] = expression
            is_valid = True
            for match in matches:
                for match in matches:
                    try:
                        is_match = match_expression(expression, match)
                    except TimeoutError:
                        # print("TimeoutError: ", expression, match)
                        is_match = None
                    if is_match is None or is_match == False:
                        is_valid = False
                        break
            for non_match in non_matches:
                try:
                    is_match = match_expression(expression, non_match)
                except TimeoutError:
                    # print("TimeoutError: ", expression, match)
                    is_match = None
                if is_match is None or is_match == True:
                    is_valid = False
                    break
            item['passed'] = is_valid
            compilation_result.append({
                'id': id,
                'completion': expression,
                'passed': is_valid
            })  
        # print("Ending item: ", item['id'])
    with open(f'./Output/{outfilename}', 'w') as f:
        json.dump(compilation_result, f, indent=4)

    